In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import load_model
from matplotlib import pyplot as plt
from sklearn import preprocessing
import pandas as pd
import tensorflow as tf
import keras

#  Import and read the charity_data.csv
import pandas as pd
# application_df = pd.read_csv("Resources/charity_data.csv")
# application_df.head()

In [3]:
parse_dates = ["GAME DATE"]
nba_df = pd.DataFrame(pd.read_csv("Data/Sample_NBA_Data_Reg_Season_2022.csv", parse_dates=parse_dates))

nba_teams_df = pd.DataFrame(pd.read_csv("nba_team_stats_data/nba_adv_stats.csv"))
nba_def_df = pd.DataFrame(pd.read_csv("nba_team_stats_data/nba_def.csv"))
nba_off_df = pd.DataFrame(pd.read_csv("nba_team_stats_data/nba_off.csv"))
nba_post_adv_stats_df = pd.DataFrame(pd.read_csv("nba_team_stats_data/nba_post_adv_stats.csv"))
nba_post_def_df = pd.DataFrame(pd.read_csv("nba_team_stats_data/nba_post_def.csv"))
nba_post_off_df = pd.DataFrame(pd.read_csv("nba_team_stats_data/nba_post_off.csv"))

reg_season_compiled = pd.DataFrame(pd.read_csv("nba_team_stats_data/nba_reg_compiled_stats.csv").drop(['Unnamed: 0'],axis=1))
post_season_compiled = pd.DataFrame(pd.read_csv("nba_team_stats_data/nba_post_compiled_stats.csv").drop(['Unnamed: 0'],axis=1))

In [4]:
team_names_series = reg_season_compiled["Team"]

team_abbv = ["MIN", "MEM", "MIL", "CHA", "PHX", "ATL", "UTA", "SAS", "BKN", "DEN", "LAL", "BOS", "CHI", "IND", "GSW", "SAC", "MIA", "PHI", "HOU", "TOR", "NOP", "WAS", "LAC", "DAL", "CLE", "NYK", "POR", "DET", "ORL", "OKC"]
team_abbv_series = pd.Series(team_abbv)

frame = { 'Team': team_names_series, 'ABBV': team_abbv_series }

name_abbv = pd.DataFrame(frame)

name_abbv

,Team,ABBV
0,Minnesota Timberwolves,MIN
1,Memphis Grizzlies,MEM
2,Milwaukee Bucks,MIL
3,Charlotte Hornets,CHA
4,Phoenix Suns,PHX
5,Atlanta Hawks,ATL
6,Utah Jazz,UTA
7,San Antonio Spurs,SAS
8,Brooklyn Nets,BKN
9,Denver Nuggets,DEN


In [5]:
reg_season_abbv = reg_season_compiled.merge(name_abbv, how = "left", on = "Team")

In [6]:
# Create "matches" for the usuable match up data
matches = nba_df[["TEAM", "MATCH UP", "W/L"]].copy()

# Split the "MATCH UP" Column into 3 separate columns
matches[["TEAM", "TYPE", "OPPONENT"]] = matches['MATCH UP'].apply(lambda x: pd.Series(x.split(' ')))

# Drop the original "MATCH UP" column
matches = matches.drop(["MATCH UP"], axis = 1)

In [7]:
# Distinguish Opponent Data from "Defending" Team Data   - Is "defending" the right term?
oppo_reg_season_abbv = reg_season_abbv.add_prefix('OPPO_')

In [8]:
# Merge Team Agg Info with Match Info, twice... Once for defending team, once for opponent
team_games_data = matches.merge(reg_season_abbv, how = "left", left_on = "TEAM", right_on = "ABBV").merge(oppo_reg_season_abbv, how = "left", left_on = "TEAM", right_on = "OPPO_ABBV")

# Drop target and superfluous columns
team_games_data = team_games_data.drop(['W/L', "Team", "OPPO_Team", "ABBV", "OPPO_ABBV"], axis = 1)

In [9]:
# Designate the target values as the winning side of W/L
y = pd.get_dummies(matches['W/L'])
y = y["W"]

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
team_games_data_dummies = pd.get_dummies(team_games_data)
team_games_data_dummies.shape

(250, 196)

In [11]:
# Split our preprocessed data into our features and target arrays
X = team_games_data_dummies
# y = team_games_dummies['W/L']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
acc = []
min = 1
max = 5
for a in range(min, max):
    for b in range(min, max):
        for c in range(min, max):
            for d in range(min, max):
                # Number of Neurons
                neurons_1 = a
                neurons_2 = b
                neurons_3 = c
                neurons_4 = d

                # Activation Types
                activation_1 = "tanh" # Original: "relu"
                activation_2 = "sigmoid"
                activation_3 = "sigmoid"
                activation_4 = "sigmoid"
                activation_outer = "sigmoid"

                #Number of Epochs
                num_epochs = 30 # Original: 50

                # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
                num_features = len(X_train_scaled[0])

                nn = tf.keras.models.Sequential()

                # First hidden layer
                nn.add(tf.keras.layers.Dense(units = neurons_1, activation = activation_1, input_dim = num_features))

                # Second hidden layer ("If Necessary")
                nn.add(tf.keras.layers.Dense(units = neurons_2, activation = activation_2))

                # Third hidden layer ("If Necessary")
                nn.add(tf.keras.layers.Dense(units = neurons_3, activation = activation_3))

                # Third hidden layer ("If Necessary")
                nn.add(tf.keras.layers.Dense(units = neurons_4, activation = activation_4))

                # Output layer
                nn.add(tf.keras.layers.Dense(units=1, activation = activation_outer))

                # Check the structure of the model
                #nn.summary()

                # Compile the model
                nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

                # Train the model
                fit_model = nn.fit(X_train_scaled, y_train, epochs=num_epochs)


                # Create a DataFrame containing training history
                history_df = pd.DataFrame(fit_model.history)

                # Increase the index by 1 to match the number of epochs
                history_df.index += 1

                # Plot the loss
                #history_df.plot(y="loss")
                #history_df.plot(y="accuracy");

                # Evaluate the model using the test data
                model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=0)
                #print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
                acc.append(model_accuracy)
                if(model_accuracy > 0.7):
                    # Save and export your results to an HDF5 file
                    accu = round(model_accuracy * 10_000)
                    nn.save(f"Models/NBA_model_loop{accu}.h5")



Epoch 1/30
6/6 [==============================] - 0s 2ms/step - loss: 0.7059 - accuracy: 0.5080
Epoch 2/30
6/6 [==============================] - 0s 2ms/step - loss: 0.7052 - accuracy: 0.5080
Epoch 3/30
6/6 [==============================] - 0s 2ms/step - loss: 0.7048 - accuracy: 0.5080
Epoch 4/30
6/6 [==============================] - 0s 2ms/step - loss: 0.7041 - accuracy: 0.5080
Epoch 5/30
6/6 [==============================] - 0s 2ms/step - loss: 0.7035 - accuracy: 0.5080
Epoch 6/30
6/6 [==============================] - 0s 2ms/step - loss: 0.7030 - accuracy: 0.5080
Epoch 7/30
6/6 [==============================] - 0s 2ms/step - loss: 0.7026 - accuracy: 0.5080
Epoch 8/30
6/6 [==============================] - 0s 2ms/step - loss: 0.7022 - accuracy: 0.5080
Epoch 9/30
6/6 [==============================] - 0s 3ms/step - loss: 0.7016 - accuracy: 0.5080
Epoch 10/30
6/6 [==============================] - 0s 2ms/step - loss: 0.7012 - accuracy: 0.5080
Epoch 11/30
6/6 [======================

KeyboardInterrupt: 

In [ ]:
new_val = set(acc)
z=list(new_val)

print("Unique values:",z)

Unique values: [0.523809552192688, 0.4761904776096344, 0.5873016119003296, 0.5555555820465088]
